### Sound Snippet Extraction
Automation scripts for extracting labeled .wav snippets with duration of 1 sec from session recordings

##### Extracting and labeling food intake snippets

In [3]:
import wave
import csv
import os
from decimal import Decimal

def split_wav_by_labels(wav_file, csv_file, output_dir):
    # Open the wav file
    with wave.open(wav_file, 'rb') as wav:
        params = wav.getparams()
        framerate = wav.getframerate()
        n_channels = wav.getnchannels()
        sampwidth = wav.getsampwidth()

        
        # Read the frames from the wav file
        frames = wav.readframes(wav.getnframes())
        
        # Create the output directory if it doesn't exist
        if not os.path.exists(output_dir):
            os.makedirs(output_dir)
        
        # Read the csv file
        with open(csv_file, 'r') as labelfile:
            reader = csv.DictReader(labelfile)
            for row in reader:
                start_time = Decimal(row['start_time'])
                end_time = Decimal(row['end_time'])
                label = row['label']
                
                # Create the output wav file
                if label in {'chewing', 'swallowing'}:
                    # Calculate the start and end frame indices
                    start_frame = int(start_time * framerate)
                    end_frame = int(end_time * framerate)
                    
                    # Extract the frames for the segment
                    segment_frames = frames[start_frame * n_channels * sampwidth:end_frame * n_channels * sampwidth]
                    output_wav_file = os.path.join(output_dir, f"{label}_{row['food_type']}_{start_time}_{end_time}.wav")
                    with wave.open(output_wav_file, 'wb') as output_wav:
                        output_wav.setparams(params)
                        output_wav.writeframes(segment_frames)
                elif label in {'others', 'resting'}:
                    i = start_time
                    while (i + 1)<= end_time:
                        start_frame = int(i * framerate)
                        end_frame = int(((i + 1)) * framerate)
                        segment_frames = frames[start_frame * n_channels * sampwidth:end_frame * n_channels * sampwidth]
                        output_wav_file = os.path.join(output_dir, f"{label}_{i}_{i + 1}.wav")
                        with wave.open(output_wav_file, 'wb') as output_wav:
                            output_wav.setparams(params)
                            output_wav.writeframes(segment_frames)
                        i += 1


In [7]:
base_dir = '/Users/jannisdaiber/Documents/Repos/github/ProjectMedicalWearables/Database'
participant = 'participant01'
session = 'session01'

raw_wav_dir = base_dir + '/raw_wavs' + '/' + participant + '/' + session + '/' + participant + '_' + session + '.wav'
label_metadata_dir =  base_dir + '/label_metadata' + '/' + participant + '_' + session + '.csv'
output_dir =  base_dir + '/snippet_wavs' + '/' + participant + '/' + session


split_wav_by_labels(raw_wav_dir, label_metadata_dir, output_dir)